In [8]:
import os
import pathlib
from pathlib import Path
import glob
import re
import cx_Oracle as oracle
import pandas as pd
import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import PolynomialFeatures
import openpyxl
import shutil

In [ ]:
def make_file_path_list():
    

    # カレントディレクトリを取得
    cwd = os.getcwd()

    # カレントディレクトリにある対象のデータフォルダ名を取得(複数ある場合に対応)
    target_folders = []
    for f in glob.glob(cwd + "\\*"):
        folder_name = os.path.split(f)[1]
        if re.findall("^\d{8}", folder_name):
            target_folders.append(folder_name)

    
    # 対象ディレクトリのパスを作成(list)
    path_list = [cwd + "\\" + target_folder + "\\*\\**" for target_folder in target_folders]
    
    # データフォルダ名のリストも作成
    folder_name_list = [target_folder for target_folder in target_folders]


    # 対象ディレクトリから全てのファイル名を取得(list)
    data_folders = []

    for path in path_list:
        data_folders.append(glob.glob(path))


    # 各データフォルダから必要なファイルを抽出し、各チャネル号機毎に5つのファイルを含んだリストを作成する
    # カレントフォルダにあるデータフォルダの個数分の長さのリストができる
    csv_lists = []
    for data_folder in data_folders:

        file_list=[]

        # ファイルリストから必要なファイルのみをフィルタリング
        for file in data_folder:
            file_list += [file]

        #各チャンネル毎で必要なcsvファイルのパスを取得し、リスト化
        CH_NO = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14']
        csv_list = []
        for ch in CH_NO:
            # JH放電容量・JT放電容量・JT-10℃放電抵抗に使用
            a = ch + '_CycleEnd'
            # JH25℃SOC50% 放電抵抗に使用
            b = ch + '_Count000005Ptn005'
            # JH-35℃SOC50% 放電抵抗に使用
            c = ch + '_Count000006Ptn006'
            # JH25℃SOC10% 放電抵抗に使用
            d = ch + '_Count000009Ptn009'
            # JM0℃SOC70% 充電抵抗に使用
            e = ch + '_Count000012Ptn012'
            # JT25℃SOC60% 放電抵抗に使用
            f = ch + '_Count000017Ptn017'
            # JT-30℃SOC15% 放電抵抗に使用
            g = ch + '_Count000020Ptn020'
            # JT-10℃SOC15% 放電抵抗に使用
            h = ch + '_Count000022Ptn022'

            i = [file for file in file_list if ('Grp' in file) and ( a in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( b in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( c in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( d in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( e in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( f in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( g in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( h in file) ]

            csv_list.append(i)

        # 各データフォルダの最初のチャネル番号を取得してcsv_listにセットで格納
        for idx, ch in enumerate(csv_list, start=1):
            if ch:
                csv_list.insert(0, idx)
                break        
        
        csv_lists.append(csv_list)
    
    csv_lists_with_folder_name = []
    for folder_name, csv_list in zip(folder_name_list, csv_lists):
        csv_lists_with_folder_name.append((folder_name, csv_list))
        
    return csv_lists_with_folder_name
    

### 処理に必要なファイルだけ取得し、移動

##### フォルダ内から使用するファイル名のリストを作成し、ファイル移動をする関数

In [28]:
def make_file_path_list(mode=1, ch_no=None, file_type=None):
    
    '''
    mode: 1 -> 必要な全てのファイルリストを返す 
          2 -> 任意のファイルのパスを返す
    file_type: 1 -> 放電容量
               2 -> 充電抵抗
               3 -> CycleEnd
               4 -> 放電抵抗25
               5 -> 放電抵抗-10
    '''
    
    # カレントディレクトリを取得
    cwd = os.getcwd()
    
    # データフォルダ名を取得
    for f in glob.glob(cwd + "\\*"):
        folder_name = os.path.split(f)[1]
        if re.findall("^\d{8}", folder_name):
            target_folder = folder_name
    
    # 対象ディレクトリのパスを作成
    path = cwd + "\\" + target_folder + "\\*\\**"
    
    # 対象ディレクトリの中に必要なファイルを格納するフォルダを作成
    
    new_dir_path = cwd +  "\\" + target_folder +'★アップ用'
    os.mkdir(new_dir_path)
    
    new_dir_path = cwd +  "\\" + target_folder +'★アップ用'+'\\Grp'
    os.mkdir(new_dir_path)
    
    
    
    # 対象ディレクトリから全てのファイル名を取得(list)
    files = glob.glob(path)

    file_list=[]

    # ファイルリストから必要なファイルのみをフィルタリング
    for file in files:
        file_list += [file]

    #各チャンネル毎で必要なcsvファイルのパスを取得し、リスト化し、対象ファイルを「処理ファイル」フォルダへ移動
    CH_NO = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14']
    csv_list = []
    for ch in CH_NO:
        a = ch + '_CapacityCyc'
        b = ch + '_Count000005Ptn005'
        c = ch + '_CycleEnd'
        d = ch + '_Count000009Ptn009'
        e = ch + '_Count000014Ptn014'
        
        #各変数毎a～eの条件に一致したファイルパスを取得し、「処理ファイル」フォルダへ移動（コピー）
        file_a = [file for file in file_list if ('Grp' in file) and ( a in file) ]
        for file in file_a:
            shutil.copy(file,new_dir_path + '/')
        
        file_b = [file for file in file_list if ('Grp' in file) and ( b in file) ]
        for file in file_b:
            shutil.copy(file,new_dir_path + '/')
            
            
        file_c = [file for file in file_list if ('Grp' in file) and ( c in file) ]
        for file in file_c:
            shutil.copy(file,new_dir_path + '/')
        
        
        file_d = [file for file in file_list if ('Grp' in file) and ( d in file) ]
        for file in file_d:
            shutil.copy(file,new_dir_path + '/')
        
        file_e = [file for file in file_list if ('Grp' in file) and ( e in file) ]
        for file in file_e:
            shutil.copy(file,new_dir_path + '/')
            print(file)
    
    if mode == 1:
        return csv_list
    elif mode == 2:
        return csv_list[ch_no-1][file_type-1]

In [29]:
make_file_path_list()

C:\Users\4076509\Desktop\信頼性RPAテスト\20220311_JT_KA208(4)_#2_ch7-ch9_KA211(4)_JT#2_ch10-ch12\220311_3024-02-Grp007_01\07_Count000014Ptn014_40A_内部抵抗_-10℃_720A×300A_8点.csv
C:\Users\4076509\Desktop\信頼性RPAテスト\20220311_JT_KA208(4)_#2_ch7-ch9_KA211(4)_JT#2_ch10-ch12\220311_3024-02-Grp007_01\08_Count000014Ptn014_40A_内部抵抗_-10℃_720A×300A_8点.csv
C:\Users\4076509\Desktop\信頼性RPAテスト\20220311_JT_KA208(4)_#2_ch7-ch9_KA211(4)_JT#2_ch10-ch12\220311_3024-02-Grp007_01\09_Count000014Ptn014_40A_内部抵抗_-10℃_720A×300A_8点.csv
C:\Users\4076509\Desktop\信頼性RPAテスト\20220311_JT_KA208(4)_#2_ch7-ch9_KA211(4)_JT#2_ch10-ch12\220311_3024-02-Grp007_01\10_Count000014Ptn014_40A_内部抵抗_-10℃_720A×300A_8点.csv
C:\Users\4076509\Desktop\信頼性RPAテスト\20220311_JT_KA208(4)_#2_ch7-ch9_KA211(4)_JT#2_ch10-ch12\220311_3024-02-Grp007_01\11_Count000014Ptn014_40A_内部抵抗_-10℃_720A×300A_8点.csv
C:\Users\4076509\Desktop\信頼性RPAテスト\20220311_JT_KA208(4)_#2_ch7-ch9_KA211(4)_JT#2_ch10-ch12\220311_3024-02-Grp007_01\12_Count000014Ptn014_40A_内部抵抗_-10℃_720A×300A

[]